# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`).

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.**

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [X] Fit a ridge regression model with multiple features.
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

## Load Data

In [19]:
import pandas as pd
import ydata_profiling
# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,...,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
23035,4,ST. ALBANS,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,...,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,SUNNYSIDE,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,...,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,SUNNYSIDE,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,...,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,WOODHAVEN,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,...,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019
23039,4,WOODSIDE,12 CONDOS - WALKUP APARTMENTS,2,1216,1161,NaN,R2,"61-05 39TH AVENUE, F5",F5,...,1.0,0.0,85.0,"15,151",854.0,1927.0,2,R2,569202,04/30/2019


In [21]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

df.tail()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,...,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
23035,4,ST. ALBANS,01 ONE FAMILY DWELLINGS,1,10965,276,NaN,A5,111-17 FRANCIS LEWIS BLVD,NaN,...,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,04/30/2019
23036,4,SUNNYSIDE,09 COOPS - WALKUP APARTMENTS,2,169,29,NaN,C6,"45-14 43RD STREET, 3C",NaN,...,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,04/30/2019
23037,4,SUNNYSIDE,10 COOPS - ELEVATOR APARTMENTS,2,131,4,NaN,D4,"50-05 43RD AVENUE, 3M",NaN,...,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,04/30/2019
23038,4,WOODHAVEN,02 TWO FAMILY DWELLINGS,1,8932,18,NaN,S2,91-10 JAMAICA AVE,NaN,...,2.0,1.0,3.0,"2,078",2200.0,1931.0,1,S2,1100000,04/30/2019
23039,4,WOODSIDE,12 CONDOS - WALKUP APARTMENTS,2,1216,1161,NaN,R2,"61-05 39TH AVENUE, F5",F5,...,1.0,0.0,85.0,"15,151",854.0,1927.0,2,R2,569202,04/30/2019


In [26]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [27]:
top10

Index(['FLUSHING-NORTH', 'UPPER EAST SIDE (59-79)', 'UPPER EAST SIDE (79-96)',
       'BEDFORD STUYVESANT', 'BOROUGH PARK', 'UPPER WEST SIDE (59-79)',
       'GRAMERCY', 'ASTORIA', 'FOREST HILLS', 'UPPER WEST SIDE (79-96)'],
      dtype='object', name='NEIGHBORHOOD')

In [29]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,...,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,...,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,...,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,...,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,...,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,...,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [30]:
# ('BUILDING_CLASS_CATEGORY' == '01 ONE FAMILY DWELLINGS')

df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df.drop('EASE-MENT', axis = 1)
df = df.drop('APARTMENT_NUMBER', axis = 1)

## Split Training and Test Data

In [31]:
# - [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
# 01 - 03 2019 train, 04 2019 test
# property sales prices

train     = df[df['SALE_DATE'].str.contains('0[1-3]/[0-3][0-9]/2019')]
test      = df[df['SALE_DATE'].str.contains('04/[0-3][0-9]/2019')]

target    = 'SALE_PRICE'
high_card = ['BLOCK', 'LOT', 'ADDRESS', 'ZIP_CODE', 'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'SALE_DATE']
features  = train.columns.drop([target] + high_card)

X_train   = train[features]
y_train   = train[target]
X_test    = test[features]
y_test    = test[target]

In [42]:
y_train.head()

,SALE_PRICE
7,0
8,0
9,0
42,0
44,550000


In [32]:
train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,A1,1193 SACKET AVENUE,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,01/01/2019
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,A5,1215 VAN NEST AVENUE,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,A5,1211 VAN NEST AVENUE,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,A1,2601 AVENUE R,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,01/01/2019
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019


In [33]:
X_train.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,A1,1.0,0.0,1.0,1,A1
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,A5,1.0,0.0,1.0,1,A5
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,A5,1.0,0.0,1.0,1,A5
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,A1,1.0,0.0,1.0,1,A1
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,A9,1.0,0.0,1.0,1,A9


## Check NA Values

In [34]:
train.isnull().sum()

,0
BOROUGH,0
NEIGHBORHOOD,0
BUILDING_CLASS_CATEGORY,0
TAX_CLASS_AT_PRESENT,0
BLOCK,0
LOT,0
BUILDING_CLASS_AT_PRESENT,0
ADDRESS,0
ZIP_CODE,0
RESIDENTIAL_UNITS,0


In [35]:
# - [X] Do one-hot encoding of categorical features.
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names = True)
X_train = encoder.fit_transform(X_train)
X_test  = encoder.transform(X_test)

In [36]:
X_train.head()

,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_BEDFORD STUYVESANT,...,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A7,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A6,BUILDING_CLASS_AT_TIME_OF_SALE_A8,BUILDING_CLASS_AT_TIME_OF_SALE_S0
7,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [37]:
X_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
BOROUGH_2,4094.0,0.085247,0.279283,0.0,0.0,0.0,0.0,1.0
BOROUGH_3,4094.0,0.180020,0.384251,0.0,0.0,0.0,0.0,1.0
BOROUGH_4,4094.0,0.486810,0.499887,0.0,0.0,0.0,1.0,1.0
BOROUGH_5,4094.0,0.240107,0.427201,0.0,0.0,0.0,0.0,1.0
BOROUGH_1,4094.0,0.007816,0.088074,0.0,0.0,0.0,0.0,1.0
NEIGHBORHOOD_OTHER,4094.0,0.934538,0.247369,0.0,1.0,1.0,1.0,1.0
NEIGHBORHOOD_FLUSHING-NORTH,4094.0,0.035906,0.186079,0.0,0.0,0.0,0.0,1.0
NEIGHBORHOOD_BOROUGH PARK,4094.0,0.007328,0.085299,0.0,0.0,0.0,0.0,1.0
NEIGHBORHOOD_UPPER EAST SIDE (59-79),4094.0,0.001954,0.044167,0.0,0.0,0.0,0.0,1.0
NEIGHBORHOOD_BEDFORD STUYVESANT,4094.0,0.003908,0.062401,0.0,0.0,0.0,0.0,1.0


In [38]:
# - [X] Do feature selection with `SelectKBest`.
from sklearn.feature_selection import f_regression, SelectKBest
selector         = SelectKBest(score_func = f_regression, k = 15)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected  = selector.transform(X_test)

In [39]:
# - [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
from sklearn import preprocessing

In [40]:
# - [X] Fit a ridge regression model with multiple features.
from sklearn.linear_model import Ridge
model = Ridge(alpha = 15)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [45]:
y_pred

array([3225047.05580502,  421150.6193711 ,  388334.28535342,
        488015.73318639,  442056.02818131,  488015.73318639,
        442056.02818131,  482003.69522606,  482003.69522606,
        482003.69522606,  488015.73318639,  488015.73318639,
        488015.73318639,  442056.02818131,  488015.73318639,
        366871.02896507,  366871.02896507,  400485.04013003,
        367668.70611235,  400485.04013003,  400485.04013003,
        501052.85207964,  367668.70611235,  700821.61746624,
        654861.91246116,  367668.70611235,  367668.70611235,
        367668.70611235,  367668.70611235,  354525.33512495,
        400485.04013003,  400485.04013003,  482786.05321473,
        400485.04013003,  354525.33512495,  394473.0021697 ,
        400485.04013003,  400485.04013003,  400485.04013003,
        400485.04013003,  400485.04013003,  367668.70611235,
        354525.33512495,  354525.33512495,  343537.23305139,
        356680.60403879,  630808.79146545,  343537.23305139,
        343537.23305139,

In [43]:
pd.DataFrame([X_train.columns, model.coef_], index=['Feature', 'Coef']).transpose()

,Feature,Coef
0,BOROUGH_2,-564890.865531
1,BOROUGH_3,-498025.751715
2,BOROUGH_4,-585556.444772
3,BOROUGH_5,-596544.546845
4,BOROUGH_1,2245017.608863
5,NEIGHBORHOOD_OTHER,-476478.491257
6,NEIGHBORHOOD_FLUSHING-NORTH,-375910.679308
7,NEIGHBORHOOD_BOROUGH PARK,-361438.369913
8,NEIGHBORHOOD_UPPER EAST SIDE (59-79),1072301.157575
9,NEIGHBORHOOD_BEDFORD STUYVESANT,-576027.899374


In [47]:
# - [X] Get mean absolute error for the test set.
import numpy as np
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

387119.5244040678

## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [52]:
# - [X] Fit a ridge regression model with multiple features.
from sklearn.linear_model import Ridge
model = Ridge(alpha = 20)
model.fit(X_train_selected, y_train)
y_pred = model.predict(X_test_selected)

In [53]:
# - [X] Get mean absolute error for the test set.
import numpy as np
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

390926.98623913346